In [1]:
from __future__ import annotations
from typing import Tuple, List, Dict, KeysView, Iterable
from random import uniform
from gurobipy import Model, tupledict, GurobiError, GRB

In [13]:
Vertex = int
Arc = Tuple[Vertex, Vertex]
Tour = List[Vertex]

class TSPIntance:
    n: int
    x: List[float] #contains x coordinates for all our vertices 
    y : List[float]
    cost: Dict[Arc, float] #contains y coordinates for all our vertices. Vertex i is formed by x[i], y[i]
    

    def __init__(self, x: List[float], y: List[float], costs:List[float]):
        assert len(x) == len(y), "nodes and costs lists must have the same length"

        self.n = len(x)
        self.x = x
        self.y = y
        self.cost = {
            (i, j): 10
            for i in self.vertices()
            for j in self.vertices()
            if i != j
        }
        self.cost[(1,2)]=-1
        self.cost[(2,3)]=-1
        self.cost[(3,1)]=-1
        

    def vertices(self) -> Iterable[Vertex]: #we define this so we can easily compute self.cost
        return range(self.n)
    
    def arcs(self) -> KeysView:
        return self.cost.keys()

    @staticmethod
    def random(n: int) -> TSPIntance:
        x = [uniform(0, 10) for _ in range(n)]
        y = [uniform(0, 10) for _ in range(n)]
        costs= [uniform(-10,10) for _ in range(n*(n-1))]
        return TSPIntance(x=x, y=y, costs=costs)

In [14]:
class TSPSolution: #this is what our sole method returns
    tour: Tour 
    cost: float

    def __init__(self, tour: Tour, **kwargs):
        assert 'cost' in kwargs or 'instance' in kwargs, \
            "You must pass the tour cost or a TSP instance to compute it"

        if 'cost' in kwargs:
            self.cost = kwargs.get('cost')
        elif 'instance' in kwargs:
            tsp = kwargs.get('instance')
            self.cost = sum(
                tsp.cost[i, j]
                for i in tour[:-1]
                for j in tour[1:]
            )

        self.tour = tour

    def __str__(self) -> str:
        return "[" + ', '.join(map(str, self.tour)) + f"] - Cost: {self.cost:.2f}"

In [15]:
class BranchAndCutIntegerSolver:
    tsp: TSPIntance
    m: Model
    x: tupledict

    def __init__(self, tsp: TSPIntance):
        self.tsp = tsp
        self.m = Model()
        self.x = self.m.addVars(self.tsp.arcs(), obj=self.tsp.cost, vtype=GRB.BINARY, name='x') #here we define Xij
        self.__build_model()

    def __build_model(self) -> None:
        self.m.addConstr(self.x.sum(0, '*') == 1) # only one outgoing  arc from source.
        self.m.addConstr(self.x.sum('*', self.tsp.n-1) == 1 ) # only one incoming  arc to sink.
        self.m.addConstrs(self.x.sum(i, '*') == self.x.sum('*',i) for i in range(1, self.tsp.n-1)) # todo lo que entra, sale
        self.m.addConstr(self.x.sum(self.tsp.n-1, '*') == 0 ) # no outgoing arcs from sink
        self.m.addConstr(self.x.sum('*', 0) == 0 ) # no incoming arcs to source
        self.m.addConstrs((self.x.sum(i, '*') <=1 for i in self.tsp.vertices())) # one outgoing arc at most
        
        
    def solve(self) -> TSPSolution:
        self.m.setParam(GRB.Param.LazyConstraints, 1) 
        self.m.optimize(lambda _, where: self.__find_subtours(where=where)) #here we call the optimize gurobi function, that solves the problem.
        #                        we don't care about the first argument of m.optimize (a Model), so we write '_' in the lambda
        #                        our second parameter is a number that tells us where we are in the solution function. 
        #                        we will use it to check when we are at a possible feasible sol, so then we check for subsets (sub_tours). 
        if self.m.Status != GRB.OPTIMAL:
            raise RuntimeError("Could not solve TSP model to optimality")
        
        return TSPSolution(tour=self.__tour_starting_at(0), cost=self.m.ObjVal)
    
    def __find_subtours(self, where: int) -> None: #here we will look for sub_tours
        if where != GRB.Callback.MIPSOL: #MIPSOL means gurobi found a possibly feasible solution, which is what we  are looking for now. 
            return
        
        remaining = set(self.tsp.vertices()) #these are the vertices we are yet to explore. at first we have all of them

        while len(remaining) > 0:
            # Get the first vertex of the set
            start = next(iter(remaining))
            subtour = self.__tour_starting_at(start)
                    
            if subtour != [start] and subtour[-1]!=self.tsp.n-1 :
                 self.__add_sec_for(subtour)
                    
            remaining -= set(subtour)

    def __tour_starting_at(self, i) -> Tour: #we start out tour at i, and keep adding vertices to our tour until we run into i again.
        tour = [i] 
        current = self.__next_vertex(i=i)

        if current is None: #for vertices that are not in optimal sol, we will find no next vertex. We then return a  list with such vertex, which will be excluded of the "remaining" set
            return tour          
            
        while current != i : #we will cut the tour whenever we reach the tour origin, or the sink
            tour.append(current)
            if current == self.tsp.n-1: #if we reach the sink, we stop
                break
            current = self.__next_vertex(current)

        return tour

    def __next_vertex(self, i: Vertex) -> Vertex: #this function moves on to another vertex during our sub_tour search. 
        for j in self.tsp.vertices():
            if j == i:
                continue
            try:
                # When in a callback
                x = self.m.cbGetSolution(self.x[i,j]) #this is our way to aquire the value of X before reaching the Solution. 
            except GurobiError:
                # When optimisation is over
                x = self.x[i,j].X  # this is the way to reach our value after we reach the optimum. 

            if x > 0.5: #gurobi makes small errors so not all x's will be exactly 1. This is why we write x>0.5 
                return j
            
        return None
    
    def __add_sec_for(self, subtour: Tour) -> None:
        print("Adding subtour for [" + ', '.join(map(str, subtour)) + "]")
        #edges = [(subtour[i], subtour[i + 1]) for i in range(len(subtour) - 1)] + [(subtour[-1], subtour[0])]
        self.m.cbLazy(
            sum(
                self.x[i, j]
                for i, j in self.tsp.arcs()
                if i in subtour and j  in subtour                  
            ) <= len(subtour)-1
        )

In [16]:
tsp = TSPIntance.random(n=19)
solver = BranchAndCutIntegerSolver(tsp=tsp)
solution = solver.solve()

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2594094 - for non-commercial use only - registered to al___@bse.eu
Optimize a model with 40 rows, 342 columns and 1026 nonzeros
Model fingerprint: 0x3b97c134
Variable types: 0 continuous, 342 integer (342 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 10.0000000
Presolve removed 4 rows and 35 columns
Presolve time: 0.01s
Presolved: 36 rows, 307 columns, 903 nonzeros
Variable types: 0 continuous, 307 integer (307 binary)
Adding subtour for [1, 2, 3]

Cutting planes:
  Lazy constraints: 1

Explored 1 nodes (0 simplex iterations) in 0.0

In [17]:
print(solution)

[0, 18] - Cost: 10.00


In [18]:
import pandas as pd

data = [{"From": i, "To": j, "Cost": cost} for (i, j), cost in tsp.cost.items()]
print(pd.DataFrame(data))


     From  To  Cost
0       0   1    10
1       0   2    10
2       0   3    10
3       0   4    10
4       0   5    10
..    ...  ..   ...
337    18  13    10
338    18  14    10
339    18  15    10
340    18  16    10
341    18  17    10

[342 rows x 3 columns]
